In [1]:
import pandas as pd

In [ ]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
from openpyxl import Workbook
from openpyxl.styles import Font
semantic_cls = pipeline(Tasks.information_extraction, 'damo/nlp_bert_relation-extraction_chinese-base-commerce')


In [3]:
def review_predict(review):
    #返回结果属性词+情感极性+情感词
    #基于hugging face hfl/chinese-roberta-wwm-ext 的模型进行训练的模型，如需训练该模型需了解Bert模型训练
    result=semantic_cls(input=review)
    return result


In [4]:
excel_data=pd.read_excel('/Users/kevin.yang/Documents/develop/modelscope/review_data/Estee Lauder_review.xlsx')
df=excel_data[['买家','初评内容']]
wb = Workbook()
ws = wb.active
ws.append(['买家','评论','正/负舆情','属性','情感词'])
row=ws[1]
font=Font(bold=True)
for cell in row:
    cell.font=font
    
for v in df.to_dict(orient='records'):
    # if v['初评内容']=="包装严密，整体气味很小，是我想要的，上一罐韩国的正好用完这一罐接上，好评":
    #     break
    review=v['初评内容']
    buyer=v['买家']
    result=review_predict(review)['spo_list']
    result.sort(key=lambda x:x[1])
    #获得ws当前行数
    row_num=ws.max_row
    #先将数据全部压入worksheet，之后再进行单元格合并
    for i, data in enumerate(result):
        ws.append([buyer,review,data[1],data[0],data[2]])
    # #将ws中内容相同的单元格合并起来
    # #需要合并的行数
    row_merge=result.__len__()
    if row_merge>1:
        #第一，第二列为买家和初评内容肯定需要被合并
        ws.merge_cells(start_row=row_num+1, start_column=1, end_row=row_num+row_merge, end_column=1)
        ws.merge_cells(start_row=row_num+1, start_column=2, end_row=row_num+row_merge, end_column=2)
        #正负舆情列 需要开始合并的行和结束合并的行
        start_merge=row_num+1
        end_merge=row_num+1
        for i in range(row_num+1,row_num+row_merge+1):
            #第一行继续执行
            if i==row_num+1:
                continue
            #如果已经到了新增数据的最后一行，直接全部合并掉
            elif i==row_num+row_merge:
                end_merge = i
                ws.merge_cells(start_row=start_merge, start_column=3, end_row=end_merge, end_column=3)
            else:
                #判断本行与上一行的正负舆情列数据是否一致，如果不一致直接合并，反之结束行往后挪
                if ws.cell(row=i-1,column=3).value==ws.cell(row=i,column=3).value:
                    end_merge=i
                else:
                    #如果不一致了，立刻合并之前的行，并重新设置新合并单元的起始行
                    ws.merge_cells(start_row=start_merge, start_column=3, end_row=end_merge, end_column=3)
                    start_merge=i

wb.save('./review_data/result_data.xlsx')


2023-07-08 18:59:03,774 - modelscope - WARNING - task information-extraction input definition is missing
/Users/kevin.yang/miniforge3/envs/modelscope/lib/python3.8/site-packages/transformers/modeling_utils.py:884: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
2023-07-08 18:59:03,839 - modelscope - WARNING - task information-extraction output keys are missing
